In [1]:
from numpy import *
import matplotlib.pyplot as plt
from sklearn.datasets import load_svmlight_file
import numpy as np
from sklearn.cross_validation import train_test_split

t_X,t_y=load_svmlight_file('a9a.t') #读取数据
X_train,X_test, y_train, y_test =train_test_split(t_X,t_y,test_size=0.2, random_state=1)#分割数据

C:\Users\lkz\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def fit( x, y, c=1, lr=0.01, batch_size=128,epoch=200):  #训练svm
    batch_size = min(batch_size, len(y))
    w = np.zeros(x.shape[1])
    b = 0.
    w,b=MBGD(x,y,batch_size,epoch,lr,c,w,b)
    #w,b=RMSProp(x,y,batch_size,epoch,lr,c,w,b)
    #w,b=AdamDelta(x,y,batch_size,epoch,lr,c,w,b)
    #w,b=Adam(x,y,batch_size,epoch,lr,c,w,b)
    return w,b

In [3]:
def predict( x,w,b, raw=False):  #预测种类
    y_pred = x.dot(w) + b
    if raw:
        return y_pred
    return np.sign(y_pred).astype(np.float32)

In [4]:
def MBGD(x,y,batch_size,epoch,lr,c,w,b):
    for _ in range(epoch):
        w *= 1 - lr
        batch = np.random.choice(x.shape[0], batch_size)
        x_batch, y_batch = x[batch], y[batch]
        err = 1 - y_batch * predict(x_batch, w, b, True)
        if np.max(err) <= 0:
            continue
        mask = err > 0
        delta = lr * c * y_batch[mask]
        w += np.mean(delta[..., None].transpose() * x_batch[mask], axis=0)
        b += np.mean(delta)
    return w,b

In [5]:
def RMSProp(x,y,batch_size,epoch,lr,c,w,b):
    yita=0.9
    G = zeros((x.shape[1], 1))
    for _ in range(epoch):
        w *= 1 - lr
        batch = np.random.choice(x.shape[0], batch_size)
        x_batch, y_batch = x[batch], y[batch]
        err = 1 - y_batch * predict(x_batch, w, b, True)
        if np.max(err) <= 0:
            continue
        mask = err > 0
        delta = lr * c * y_batch[mask]
        g =delta[..., None].transpose() * x_batch[mask]
        G = yita * G +(1-yita)*multiply(g,g)
        w +=np.mean(multiply(1/np.sqrt(G+1e-8),g),axis=0)
        b += np.mean(delta)
    return w,b

In [6]:
def AdamDelta(x,y,batch_size,epoch,lr,c,w,b):
    yita=0.99
    Gt = zeros((x.shape[1], 1))
    deltat = zeros((x.shape[1], 1))
    for _ in range(epoch):
        w *= 1 - lr
        batch = np.random.choice(x.shape[0], batch_size)
        x_batch, y_batch = x[batch], y[batch]
        err = 1 - y_batch * predict(x_batch, w, b, True)
        if np.max(err) <= 0:
            continue
        mask = err > 0
        delta =  c * y_batch[mask]
        g = delta[..., None].transpose() * x_batch[mask]
        Gt = Gt + (1 - yita) * multiply(g, g)
        deltaWeight = np.mean(-multiply((np.sqrt(deltat + 1e-6) / np.sqrt(Gt + 1e-6)), g),axis=0)
        deltat = yita * deltat + (1 - yita) * multiply(deltaWeight, deltaWeight)
        w +=deltaWeight
        b += np.mean(delta)
    return w,b

In [7]:
def Adam(x,y,batch_size,epoch,lr,c,w,b):
    beta1=0.9
    beta=0.9
    yita=0.999
    mt = zeros((x.shape[1], 1))
    Gt = zeros((x.shape[1], 1))
    for t in range(epoch):
        w *= 1 - lr
        batch = np.random.choice(x.shape[0], batch_size)
        x_batch, y_batch = x[batch], y[batch]
        err = 1 - y_batch * predict(x_batch, w, b, True)
        if np.max(err) <= 0:
            continue
        mask = err > 0
        delta = c * y_batch[mask]
        g = delta[..., None].transpose() * x_batch[mask]
        mt = beta * mt + (1 - beta) * g
        Gt = Gt + (1 - yita) * multiply(g, g)
        alpha = lr * math.sqrt(1 - yita ** t) / (1 - beta1 ** (t + 1))
        w += np.mean( alpha * mt / np.sqrt(Gt + 1e-8), axis=0)
        b += np.mean(delta)
    return w,b

In [8]:
w,b=fit(X_train,y_train)

print("准确率：{:8.6} %".format((predict(X_test,w,b) == y_test).mean() * 100))

准确率： 80.8413 %
